In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import torch 
import torch.nn as nn
import yfinance as yf
import keras 
import tensorflow
import datetime 
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
def TickerIntoTensor(ticker,daysago,split_percentage):
    df = yf.Ticker(ticker.upper())
    df = df.history(period='max')
    df = df.drop(columns=['Open','High','Low','Volume','Dividends','Stock Splits'])
    for i in range(daysago,-2,-1):
        df['{}daysago'.format(i)] = df['Close'].shift(i)
        df = df.copy()
    df = df.drop(columns=['Close'])
    df = df.drop(index=df.tail(1).index)
    df = df.drop(index=df.head(daysago).index)
    sc = MinMaxScaler(feature_range=(0,1))
    df_scaled = sc.fit_transform(df)
    X,Y = df_scaled[:,:daysago+1],df_scaled[:,-1]
    split = int(len(X)*split_percentage)
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
    X_train, X_test, Y_train, Y_test = X_train.reshape((-1,daysago+1,1)), X_test.reshape((-1,daysago+1,1)), Y_train.reshape((-1,1)), Y_test.reshape((-1,1))
    X_train, X_test, Y_train, Y_test = torch.tensor(X_train).float(), torch.tensor(X_test).float(), torch.tensor(Y_train).float(), torch.tensor(Y_test).float()
    return X_train, X_test, Y_train, Y_test 



In [3]:
class TensorIntoDataset(Dataset):
    def __init__(self,a,b):
        self.a = a
        self.b = b
    def __len__(self):
        return len(self.a)
    def __getitem__(self,i):
        return self.a[i],self.b[i]

def TensorIntoDataLoader(ticker='aapl',daysago=100,split_percentage=0.8,batchsize=15):
    e = TickerIntoTensor(ticker,daysago,split_percentage)  
    train_dataset = TensorIntoDataset(e[0],e[2])
    test_dataset = TensorIntoDataset(e[1],e[3])
    train_loader = DataLoader(train_dataset, batch_size = batchsize, shuffle = True) 
    test_loader = DataLoader(test_dataset, batch_size = batchsize, shuffle = False)
    return train_loader,test_loader

TensorIntoDataLoader('aapl',100,0.85,20)
#for _, batch in enumerate(train_loader):
#    x_batch, y_batch = batch[0].to(device),batch[1].to(device)
#    print(x_batch.shape, y_batch.shape)
#    break

ListOfTickers = ['aapl','goog','^gspc']
ListOfTickersData = []
for ticker in ListOfTickers:
    e = TensorIntoDataLoader(ticker)
    ListOfTickersData.append(e)

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out